In [1]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
# Import train_test_split function
from sklearn.model_selection import train_test_split
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

import numpy as np
import scipy as sp
import pandas as pd
from IPython.display import display, HTML

In [2]:
# Load in the data
dataset = pd.read_csv('incidenttrain.csv')

In [3]:
dataset.head()

site_id                    cta_sites incident_date   week_day  week_code  \
0       14    Blue Line Belmont Station     1/31/2020     Friday          1   
1        5       61st Street Lower Yard      1/1/2020  Wednesday          2   
2       12    Blue Line Addison Station      1/1/2020  Wednesday          2   
3      143             Skokie Rail Yard      1/2/2020   Thursday          3   
4       15  Blue Line Californa Station      1/2/2020   Thursday          3   

   weekend  holiday incident_time  incident_time_1 time_category  ...  \
0        0        0          3:37                3    late_night  ...   
1        0        1          2:00                2    late_night  ...   
2        0        1          3:00                3    late_night  ...   
3        0        0          3:00                3    late_night  ...   
4        0        0         22:00               22         night  ...   

      moonset Temperature  weather_description  weather_code  Humidity  \
0  No moonset          32             Overcast             1        76   
1       23:58          28        Partly cloudy             2        82   
2       23:58          28        Partly cloudy             2        83   
3  No moonset          43        Partly cloudy             2        76   
4  No moonset          43        Partly cloudy             2        90   

   cloudcover  heatindex  hours_after_sunrise hours_to_sunset incident  
0         100         32                -4.45           14.45        1  
1          30         28                -6.30           15.50        0  
2          28         28                -5.30           14.50        0  
3          74         43                -5.30           14.52        1  
4          64         43                13.70           -4.48        0  

[5 rows x 31 columns]

In [4]:
dataset.dtypes

site_id                  int64
cta_sites               object
incident_date           object
week_day                object
week_code                int64
weekend                  int64
holiday                  int64
incident_time           object
incident_time_1          int64
time_category           object
time_code                int64
incident_category       object
atm                      int64
restaurant               int64
avgtemp                  int64
maxtemp                  int64
totalsnow              float64
sunhour                float64
sunset                  object
sunrise                 object
moonrise                object
moonset                 object
Temperature              int64
weather_description     object
weather_code             int64
Humidity                 int64
cloudcover               int64
heatindex                int64
hours_after_sunrise    float64
hours_to_sunset        float64
incident                 int64
dtype: object

# UNDERSTANDING THE DATA

In [5]:
dataset.shape

(516, 31)

In [6]:
#print(dataset.dtypes)

In [7]:
dataset=dataset.drop('cta_sites',axis=1)
dataset=dataset.drop('incident_date',axis=1)
dataset=dataset.drop('week_code',axis=1)
dataset=dataset.drop('time_code',axis=1)

dataset=dataset.drop('moonrise',axis=1)
dataset=dataset.drop('sunrise',axis=1)
dataset=dataset.drop('moonset',axis=1)
dataset=dataset.drop('weather_code',axis=1)
#dataset=dataset.drop('weather_description',axis=1)
#dataset=dataset.drop('time_category',axis=1)
dataset=dataset.drop('incident_time',axis=1)
#dataset=dataset.drop('week_day',axis=1)
dataset=dataset.drop('sunset',axis=1)

#dataset=dataset.drop('sunset',axis=1)


In [8]:
new=dataset.copy(deep=True)

In [9]:
dataset=dataset.drop('incident_category',axis=1)


In [10]:
dataset= dataset.drop('incident_time_1',1)

In [11]:
dataset.shape

(516, 19)

In [12]:
dataset.head()

site_id   week_day  weekend  holiday time_category  atm  restaurant  \
0       14     Friday        0        0    late_night    0           0   
1        5  Wednesday        0        1    late_night    0           0   
2       12  Wednesday        0        1    late_night    0           0   
3      143   Thursday        0        0    late_night    0           0   
4       15   Thursday        0        0         night    1           1   

   avgtemp  maxtemp  totalsnow  sunhour  Temperature weather_description  \
0       32       34       0.12      5.0           32            Overcast   
1       34       39       0.00      8.7           28       Partly cloudy   
2       34       39       0.00      8.7           28       Partly cloudy   
3       41       45       0.00      5.2           43       Partly cloudy   
4       41       45       0.00      5.2           43       Partly cloudy   

   Humidity  cloudcover  heatindex  hours_after_sunrise  hours_to_sunset  \
0        76         100         32                -4.45            14.45   
1        82          30         28                -6.30            15.50   
2        83          28         28                -5.30            14.50   
3        76          74         43                -5.30            14.52   
4        90          64         43                13.70            -4.48   

   incident  
0         1  
1         0  
2         0  
3         1  
4         0

In [13]:
dummy=pd.get_dummies(dataset['week_day'])
dummy2=pd.get_dummies(dataset['time_category'])
dummy3=pd.get_dummies(dataset['weather_description'])

In [14]:
dummy.head()

Friday  Monday  Saturday  Sunday  Thursday  Tuesday  Wednesday
0       1       0         0       0         0        0          0
1       0       0         0       0         0        0          1
2       0       0         0       0         0        0          1
3       0       0         0       0         1        0          0
4       0       0         0       0         1        0          0

In [15]:
dataset=pd.concat([dataset,dummy3],axis=1)
dataset=pd.concat([dataset,dummy2],axis=1)
dataset=pd.concat([dataset,dummy],axis=1)

In [16]:
dataset.head(5)

site_id   week_day  weekend  holiday time_category  atm  restaurant  \
0       14     Friday        0        0    late_night    0           0   
1        5  Wednesday        0        1    late_night    0           0   
2       12  Wednesday        0        1    late_night    0           0   
3      143   Thursday        0        0    late_night    0           0   
4       15   Thursday        0        0         night    1           1   

   avgtemp  maxtemp  totalsnow  ...  late_night  morning night  Friday  \
0       32       34       0.12  ...           1        0     0       1   
1       34       39       0.00  ...           1        0     0       0   
2       34       39       0.00  ...           1        0     0       0   
3       41       45       0.00  ...           1        0     0       0   
4       41       45       0.00  ...           0        0     1       0   

   Monday  Saturday  Sunday  Thursday  Tuesday  Wednesday  
0       0         0       0         0        0          0  
1       0         0       0         0        0          1  
2       0         0       0         0        0          1  
3       0         0       0         1        0          0  
4       0         0       0         1        0          0  

[5 rows x 60 columns]

In [17]:
dataset=dataset.drop('week_day',axis=1)
dataset=dataset.drop('time_category',axis=1)
dataset=dataset.drop('weather_description',axis=1)

In [18]:
#dataset.dtypes

In [19]:
newdata=dataset.copy(deep=True)
expdata=dataset.copy(deep=True)

# Correlation 

In [20]:
cor = dataset.corr()
#Correlation with output variable
cor_target = abs(cor["incident"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0]
#print('\nSelected features by Filter model:\n',relevant_features)

In [21]:
relevant_features

site_id                            0.176599
weekend                            0.071999
holiday                            0.034042
atm                                0.018673
restaurant                         0.138854
avgtemp                            0.017224
maxtemp                            0.029835
totalsnow                          0.025669
sunhour                            0.039831
Temperature                        0.041723
Humidity                           0.088539
cloudcover                         0.052930
heatindex                          0.048272
hours_after_sunrise                0.033825
hours_to_sunset                    0.036965
incident                           1.000000
Blizzard                           0.030387
Blowing snow                       0.024281
Clear                              0.092132
Cloudy                             0.017267
Heavy rain                         0.056319
Heavy snow                         0.048758
Ice pellets                     

# KNN

# KNN require 2 basic pre requisites
1. Normalized data
2. All columns must be numerical

In [31]:
data_knn=dataset.copy(deep=True) 


In [32]:
# Normalizing 
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
cols_numeric = data_knn.select_dtypes(include=numerics).columns.tolist()
print('Selected numerical columns:\n',cols_numeric) 

Selected numerical columns:
 ['site_id', 'weekend', 'holiday', 'atm', 'restaurant', 'avgtemp', 'maxtemp', 'totalsnow', 'sunhour', 'Temperature', 'Humidity', 'cloudcover', 'heatindex', 'hours_after_sunrise', 'hours_to_sunset', 'incident']


In [33]:
#Using Min-Max Normalization to scale [0, 1]
for col in cols_numeric:
    data_knn[col]=(data_knn[col]-data_knn[col].min())/(data_knn[col].max()-data_knn[col].min())

In [34]:
display(HTML(data_knn.head(10).to_html()))

In [26]:
#print(data_knn['incident'])

In [52]:
# Build KNN Model # Building and Evaluating Models
# N-fold cross Validation 

from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns
from sklearn import neighbors
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# KNN requires Label so we need to Pre-Process the Label values

from sklearn import preprocessing
from sklearn.model_selection import cross_val_score

x_train, x_test, y_train, y_test = train_test_split(data_knn, y_encoded, test_size=0.2)

x = data_knn.drop('incident',1)
y = data_knn['incident'] 


for k in range(1, 24, 2): 
    clf=neighbors.KNeighborsClassifier(k, weights='uniform')
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print('K =', k, ', Accuracy: ', accuracy_score(y_test, y_pred), ', Precision: ', precision_score(y_test, y_pred, average='micro'),
         ', Recall: ', recall_score(y_test, y_pred, average='micro'))
 

K = 1 , Accuracy:  0.7596153846153846 , Precision:  0.7596153846153846 , Recall:  0.7596153846153846
K = 3 , Accuracy:  0.7596153846153846 , Precision:  0.7596153846153846 , Recall:  0.7596153846153846
K = 5 , Accuracy:  0.7884615384615384 , Precision:  0.7884615384615384 , Recall:  0.7884615384615384
K = 7 , Accuracy:  0.7884615384615384 , Precision:  0.7884615384615384 , Recall:  0.7884615384615384
K = 9 , Accuracy:  0.8076923076923077 , Precision:  0.8076923076923077 , Recall:  0.8076923076923077
K = 11 , Accuracy:  0.7788461538461539 , Precision:  0.7788461538461539 , Recall:  0.7788461538461539
K = 13 , Accuracy:  0.8076923076923077 , Precision:  0.8076923076923077 , Recall:  0.8076923076923077
K = 15 , Accuracy:  0.7884615384615384 , Precision:  0.7884615384615384 , Recall:  0.7884615384615384
K = 17 , Accuracy:  0.7980769230769231 , Precision:  0.7980769230769231 , Recall:  0.7980769230769231
K = 19 , Accuracy:  0.8076923076923077 , Precision:  0.8076923076923077 , Recall:  0.80

In [53]:
### Best Accuracy is 83.65 at K =23

In [54]:
dataset.head(5)

site_id  weekend  holiday  atm  restaurant  avgtemp  maxtemp  totalsnow  \
0       14        0        0    0           0       32       34       0.12   
1        5        0        1    0           0       34       39       0.00   
2       12        0        1    0           0       34       39       0.00   
3      143        0        0    0           0       41       45       0.00   
4       15        0        0    1           1       41       45       0.00   

   sunhour  Temperature  ...  late_night  morning  night  Friday  Monday  \
0      5.0           32  ...           1        0      0       1       0   
1      8.7           28  ...           1        0      0       0       0   
2      8.7           28  ...           1        0      0       0       0   
3      5.2           43  ...           1        0      0       0       0   
4      5.2           43  ...           0        0      1       0       0   

   Saturday  Sunday  Thursday  Tuesday  Wednesday  
0         0       0         0        0          0  
1         0       0         0        0          1  
2         0       0         0        0          1  
3         0       0         1        0          0  
4         0       0         1        0          0  

[5 rows x 57 columns]

# Naive Bayes

In [79]:
# Requirements: 1)Nominal features 2)Assumption of conditionally indepenence
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score

In [80]:
data_nb=dataset.copy(deep=True) 

In [81]:
data_nb.columns

Index(['site_id', 'weekend', 'holiday', 'atm', 'restaurant', 'avgtemp',
       'maxtemp', 'totalsnow', 'sunhour', 'Temperature', 'Humidity',
       'cloudcover', 'heatindex', 'hours_after_sunrise', 'hours_to_sunset',
       'incident', 'Blizzard', 'Blowing snow', 'Clear', 'Cloudy', 'Heavy rain',
       'Heavy snow', 'Ice pellets', 'Light drizzle', 'Light rain',
       'Light rain shower', 'Light sleet', 'Light snow', 'Light snow showers',
       'Mist', 'Moderate or heavy freezing rain', 'Moderate or heavy sleet',
       'Moderate or heavy snow showers', 'Moderate rain', 'Moderate snow',
       'Overcast', 'Partly cloudy', 'Patchy heavy snow',
       'Patchy light drizzle', 'Patchy light snow', 'Patchy moderate snow',
       'Patchy rain possible', 'Patchy snow possible', 'Sunny', 'afternoon',
       'early_morning', 'evening', 'late_night', 'morning', 'night', 'Friday',
       'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday'],
      dtype='object')

In [82]:
data_nb['site_id'] = pd.cut(data_nb['site_id'],30)
#data_nb['week_code'] = pd.cut(data_nb['week_code'],3)
data_nb['weekend'] = pd.cut(data_nb['weekend'],2)
data_nb['holiday'] = pd.cut(data_nb['holiday'],2)
#data_nb['incident_time.1'] = pd.cut(data_nb['incident_time.1'],3)
#data_nb['time_code'] = pd.cut(data_nb['time_code'],2)
data_nb['atm'] = pd.cut(data_nb['atm'],3)
data_nb['restaurant'] = pd.cut(data_nb['restaurant'],3)
data_nb['avgtemp'] = pd.cut(data_nb['avgtemp'],2)
data_nb['maxtemp'] = pd.cut(data_nb['maxtemp'],2)
data_nb['totalsnow'] = pd.cut(data_nb['totalsnow'],2)
data_nb['sunhour'] = pd.cut(data_nb['sunhour'],2)
data_nb['Temperature'] = pd.cut(data_nb['Temperature'],3)
data_nb['Humidity'] = pd.cut(data_nb['Humidity'],2)
data_nb['cloudcover'] = pd.cut(data_nb['cloudcover'],2)
data_nb['heatindex'] = pd.cut(data_nb['heatindex'],2)
data_nb['hours_after_sunrise'] = pd.cut(data_nb['hours_after_sunrise'],3)
data_nb['hours_to_sunset'] = pd.cut(data_nb['hours_to_sunset'],3)
#data_nb['weather_code'] = pd.cut(data_nb['weather_code'],3)
#data_nb['incident'] = pd.cut(data_nb['incident'],2)

In [83]:
#data_nb.columns

In [84]:
data_nb[['Blizzard', 'Blowing snow', 'Clear', 'Cloudy', 'Heavy rain',
       'Heavy snow', 'Ice pellets', 'Light drizzle', 'Light rain',
       'Light rain shower', 'Light sleet', 'Light snow', 'Light snow showers',
       'Mist', 'Moderate or heavy freezing rain', 'Moderate or heavy sleet',
       'Moderate or heavy snow showers', 'Moderate rain', 'Moderate snow',
       'Overcast', 'Partly cloudy', 'Patchy heavy snow',
       'Patchy light drizzle', 'Patchy light snow', 'Patchy moderate snow',
       'Patchy rain possible', 'Patchy snow possible', 'Sunny', 'afternoon',
       'early_morning', 'evening', 'late_night', 'morning', 'night', 'Friday',
       'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']]=data_nb[['Blizzard', 'Blowing snow', 'Clear', 'Cloudy', 'Heavy rain',
       'Heavy snow', 'Ice pellets', 'Light drizzle', 'Light rain',
       'Light rain shower', 'Light sleet', 'Light snow', 'Light snow showers',
       'Mist', 'Moderate or heavy freezing rain', 'Moderate or heavy sleet',
       'Moderate or heavy snow showers', 'Moderate rain', 'Moderate snow',
       'Overcast', 'Partly cloudy', 'Patchy heavy snow',
       'Patchy light drizzle', 'Patchy light snow', 'Patchy moderate snow',
       'Patchy rain possible', 'Patchy snow possible', 'Sunny', 'afternoon',
       'early_morning', 'evening', 'late_night', 'morning', 'night', 'Friday',
       'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']].astype('category')

In [85]:
data_nb.dtypes

site_id                            category
weekend                            category
holiday                            category
atm                                category
restaurant                         category
avgtemp                            category
maxtemp                            category
totalsnow                          category
sunhour                            category
Temperature                        category
Humidity                           category
cloudcover                         category
heatindex                          category
hours_after_sunrise                category
hours_to_sunset                    category
incident                              int64
Blizzard                           category
Blowing snow                       category
Clear                              category
Cloudy                             category
Heavy rain                         category
Heavy snow                         category
Ice pellets                     

In [87]:
y=data_nb['incident']
le = preprocessing.LabelEncoder()
le.fit(y)
y_encoded = le.transform(y)


In [89]:
data_nb['incident']=y_encoded
display(HTML(data_nb.head(5).to_html()))

In [86]:




# by N-fold evaluation ###########################################################
y=data_nb['incident']
x=data_nb.drop('incident',axis=1)
clf = GaussianNB()
acc=cross_val_score(clf, x, y, cv=5, scoring='accuracy').mean()
print("Accuracy by N-fold Cross Validation:",acc)

C:\Users\ashag\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


TypeError: float() argument must be a string or a number, not 'pandas._libs.interval.Interval'

In [431]:
#y=dataset['incident']
#le = preprocessing.LabelEncoder()
#le.fit(y)
#y_encoded = le.transform(y)

In [433]:
baseline_accuracy=0.58813326

In [434]:
dataset.shape

(516, 57)

In [435]:
data_nb.shape

(516, 144)

# DECISON TREE AND RANDOM FOREST

In [436]:
# 04. Decision Trees and Random Forest 
# Preprocessing: 1)Encoding the labels; 2)Convert Numerical to Categorical

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

In [437]:
# N-fold cross Evaluation
acc=cross_val_score(clf, x, y, cv=5, scoring='accuracy').mean()
print("Tree Accuracy by N-fold Cross Validation:",acc)

tree = DecisionTreeClassifier()
bag = BaggingClassifier(tree, n_estimators=100, max_samples=0.8, random_state=1)
acc=cross_val_score(bag, x, y, cv=5, scoring='accuracy').mean()
print("RandomForest Accuracy by N-fold Cross Validation:",acc)

Tree Accuracy by N-fold Cross Validation: 0.5813326450819312
RandomForest Accuracy by N-fold Cross Validation: 0.815692864150888


########

# Complex Algorithm:

In [438]:
dataset=data_knn.copy(deep=True)

In [439]:
print(dataset.head(5))

    site_id  weekend  holiday  atm  restaurant   avgtemp   maxtemp  totalsnow  \
0  0.086667      0.0      0.0  0.0        0.00  0.560976  0.487805   0.034682   
1  0.026667      0.0      1.0  0.0        0.00  0.609756  0.609756   0.000000   
2  0.073333      0.0      1.0  0.0        0.00  0.609756  0.609756   0.000000   
3  0.946667      0.0      0.0  0.0        0.00  0.780488  0.756098   0.000000   
4  0.093333      0.0      0.0  0.2        0.05  0.780488  0.756098   0.000000   

    sunhour  Temperature  ...  late_night  morning  night  Friday  Monday  \
0  0.197531     0.551020  ...           1        0      0       1       0   
1  0.654321     0.469388  ...           1        0      0       0       0   
2  0.654321     0.469388  ...           1        0      0       0       0   
3  0.222222     0.775510  ...           1        0      0       0       0   
4  0.222222     0.775510  ...           0        0      1       0       0   

   Saturday  Sunday  Thursday  Tuesday  Wednesday 

In [440]:
import numpy as np
import scipy.stats

In [441]:
dataset.columns

Index(['site_id', 'weekend', 'holiday', 'atm', 'restaurant', 'avgtemp',
       'maxtemp', 'totalsnow', 'sunhour', 'Temperature', 'Humidity',
       'cloudcover', 'heatindex', 'hours_after_sunrise', 'hours_to_sunset',
       'incident', 'Blizzard', 'Blowing snow', 'Clear', 'Cloudy', 'Heavy rain',
       'Heavy snow', 'Ice pellets', 'Light drizzle', 'Light rain',
       'Light rain shower', 'Light sleet', 'Light snow', 'Light snow showers',
       'Mist', 'Moderate or heavy freezing rain', 'Moderate or heavy sleet',
       'Moderate or heavy snow showers', 'Moderate rain', 'Moderate snow',
       'Overcast', 'Partly cloudy', 'Patchy heavy snow',
       'Patchy light drizzle', 'Patchy light snow', 'Patchy moderate snow',
       'Patchy rain possible', 'Patchy snow possible', 'Sunny', 'afternoon',
       'early_morning', 'evening', 'late_night', 'morning', 'night', 'Friday',
       'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday'],
      dtype='object')

In [442]:
dataset.head()

site_id  weekend  holiday  atm  restaurant   avgtemp   maxtemp  totalsnow  \
0  0.086667      0.0      0.0  0.0        0.00  0.560976  0.487805   0.034682   
1  0.026667      0.0      1.0  0.0        0.00  0.609756  0.609756   0.000000   
2  0.073333      0.0      1.0  0.0        0.00  0.609756  0.609756   0.000000   
3  0.946667      0.0      0.0  0.0        0.00  0.780488  0.756098   0.000000   
4  0.093333      0.0      0.0  0.2        0.05  0.780488  0.756098   0.000000   

    sunhour  Temperature  ...  late_night  morning  night  Friday  Monday  \
0  0.197531     0.551020  ...           1        0      0       1       0   
1  0.654321     0.469388  ...           1        0      0       0       0   
2  0.654321     0.469388  ...           1        0      0       0       0   
3  0.222222     0.775510  ...           1        0      0       0       0   
4  0.222222     0.775510  ...           0        0      1       0       0   

   Saturday  Sunday  Thursday  Tuesday  Wednesday  
0         0       0         0        0          0  
1         0       0         0        0          1  
2         0       0         0        0          1  
3         0       0         1        0          0  
4         0       0         1        0          0  

[5 rows x 57 columns]

# Bagged Decision Tree

In [444]:
for label in dataset.columns:
    dataset[label] = LabelEncoder().fit(dataset[label]).transform(dataset[label])

In [445]:
#Display information about the data set
#print(dataset.info())

In [446]:
X = dataset.drop(['incident'], axis=1)
Y = dataset['incident']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [447]:
model_DT = DecisionTreeClassifier(criterion='entropy', max_depth=1)
AdaBoost = AdaBoostClassifier(base_estimator=model_DT, n_estimators=400, learning_rate=1)

In [448]:
#Fit the model with training data
boostmodel = AdaBoost.fit(X_train, Y_train)

In [449]:
#Evaluate the accuracy of the model
y_pred = boostmodel.predict(X_test)
predictions = metrics.accuracy_score(Y_test, y_pred)
#Calculating the accuracy in percentage
print('The accuracy is: ', predictions * 100, '%')


The accuracy is:  87.09677419354838 %


# Extension- DT Bagged Random Forest

In [450]:
import pandas
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

In [451]:
seed = 7
num_trees = 100
max_features = 3
kfold = model_selection.KFold(n_splits=5, random_state=seed)
model_RF = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = model_selection.cross_val_score(model_RF, X, Y, cv=kfold)
print(results.mean())

0.7035660941000748


# Extra tree Classification - Extended Random Forest

In [452]:
from sklearn import model_selection
from sklearn.ensemble import ExtraTreesClassifier

In [453]:
seed = 7
num_trees = 100
max_features = 7
kfold = model_selection.KFold(n_splits=5, random_state=seed)
model_ET = ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features)
results = model_selection.cross_val_score(model_ET, X, Y, cv=kfold)
print(results.mean())

0.7151232262882748


# Boosting Algorithm

Boosting ensemble algorithms creates a sequence of models that attempt to correct the mistakes of the models before them in the sequence.

# Adaboost

In [454]:

from sklearn import model_selection
from sklearn.ensemble import AdaBoostClassifier
seed = 7
num_trees = 30
kfold = model_selection.KFold(n_splits=5, random_state=seed)
model_AB = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model_AB, X, Y, cv=kfold)
print(results.mean())

0.7965272591486184


# Stochastic Gradient

In [455]:
from sklearn import model_selection
from sklearn.ensemble import GradientBoostingClassifier

In [456]:
seed = 7
num_trees = 100
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model_SG = GradientBoostingClassifier(n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model_SG, X, Y, cv=kfold)
print(results.mean())

0.8603695324283558


In [457]:
new.columns

Index(['site_id', 'week_day', 'weekend', 'holiday', 'incident_time_1',
       'time_category', 'incident_category', 'atm', 'restaurant', 'avgtemp',
       'maxtemp', 'totalsnow', 'sunhour', 'Temperature', 'weather_description',
       'Humidity', 'cloudcover', 'heatindex', 'hours_after_sunrise',
       'hours_to_sunset', 'incident'],
      dtype='object')

In [458]:
dummy=pd.get_dummies(new['week_day'])
dummy2=pd.get_dummies(new['time_category'])
dummy3=pd.get_dummies(new['weather_description'])

In [459]:
new=pd.concat([new,dummy3],axis=1)
new=pd.concat([new,dummy2],axis=1)
new=pd.concat([new,dummy],axis=1)

In [460]:
new.columns

Index(['site_id', 'week_day', 'weekend', 'holiday', 'incident_time_1',
       'time_category', 'incident_category', 'atm', 'restaurant', 'avgtemp',
       'maxtemp', 'totalsnow', 'sunhour', 'Temperature', 'weather_description',
       'Humidity', 'cloudcover', 'heatindex', 'hours_after_sunrise',
       'hours_to_sunset', 'incident', 'Blizzard', 'Blowing snow', 'Clear',
       'Cloudy', 'Heavy rain', 'Heavy snow', 'Ice pellets', 'Light drizzle',
       'Light rain', 'Light rain shower', 'Light sleet', 'Light snow',
       'Light snow showers', 'Mist', 'Moderate or heavy freezing rain',
       'Moderate or heavy sleet', 'Moderate or heavy snow showers',
       'Moderate rain', 'Moderate snow', 'Overcast', 'Partly cloudy',
       'Patchy heavy snow', 'Patchy light drizzle', 'Patchy light snow',
       'Patchy moderate snow', 'Patchy rain possible', 'Patchy snow possible',
       'Sunny', 'afternoon', 'early_morning', 'evening', 'late_night',
       'morning', 'night', 'Friday', 'Monday', 

In [461]:
#dataset=dataset.drop('cta_sites',axis=1)
#dataset=dataset.drop('incident_date',axis=1)
#dataset= dataset.drop('incident_time',1)
#dataset= dataset.drop('incident_time.1',1)
#dataset=dataset.drop('moonrise',axis=1)
#dataset=dataset.drop('sunrise',axis=1)
#dataset=dataset.drop('incident_type',axis=1)
#dataset=dataset.drop('moonset',axis=1)
new=new.drop('weather_description',axis=1)
new=new.drop('time_category',axis=1)
#dataset=dataset.drop('incident_time',axis=1)
new=new.drop('week_day',axis=1)
new=new.drop('heatindex',axis=1)
new=new.drop('Humidity',axis=1)
new=new.drop('cloudcover',axis=1)
new=new.drop('maxtemp',axis=1)
new=new.drop('avgtemp',axis=1)
#dataset=dataset.drop('Temperature',axis=1)
new=new.drop('atm',axis=1)
new=new.drop('restaurant',axis=1)
new=new.drop('sunhour',axis=1)
new=new.drop('hours_after_sunrise',axis=1)
new=new.drop('hours_to_sunset',axis=1)
new=new.drop('totalsnow',axis=1)
#dataset=dataset.drop('sunset',axis=1)
#dataset=dataset.drop('incident_category',axis=1)
#dataset=dataset.drop('moonset',axis=1)

In [462]:
new.head()

site_id  weekend  holiday  incident_time_1 incident_category  Temperature  \
0       14        0        0                3      Disturbance            32   
1        5        0        1                2              none           28   
2       12        0        1                3              none           28   
3      143        0        0                3          Accident           43   
4       15        0        0               22              none           43   

   incident  Blizzard  Blowing snow  Clear  ...  late_night  morning  night  \
0         1         0             0      0  ...           1        0      0   
1         0         0             0      0  ...           1        0      0   
2         0         0             0      0  ...           1        0      0   
3         1         0             0      0  ...           1        0      0   
4         0         0             0      0  ...           0        0      1   

   Friday  Monday  Saturday  Sunday  Thursday  Tuesday  Wednesday  
0       1       0         0       0         0        0          0  
1       0       0         0       0         0        0          1  
2       0       0         0       0         0        0          1  
3       0       0         0       0         1        0          0  
4       0       0         0       0         1        0          0  

[5 rows x 48 columns]

In [463]:
data2=new.copy(deep=True)
data2=data2.groupby("incident_category")["incident_category"].count()
#data2.order(ascending=False, inplace=True)
data2

incident_category
Access Control              11
Accident                    17
Alarms                       3
Assault                     24
Disturbance                182
Elevator issue               2
Emergency                   21
Equipment Misuse             1
General                     55
Hazards                      2
Maintenance                  3
Motor Vehicle Vandalism      3
Theft                       17
Tresspass                   13
Unsecure element            10
Weapons found                1
none                       151
Name: incident_category, dtype: int64

In [464]:
dummydf=pd.get_dummies(new,columns=["incident_category"])

In [465]:
dummydf=dummydf.join(new[["incident_category"]])

In [466]:
dummydf.columns

Index(['site_id', 'weekend', 'holiday', 'incident_time_1', 'Temperature',
       'incident', 'Blizzard', 'Blowing snow', 'Clear', 'Cloudy', 'Heavy rain',
       'Heavy snow', 'Ice pellets', 'Light drizzle', 'Light rain',
       'Light rain shower', 'Light sleet', 'Light snow', 'Light snow showers',
       'Mist', 'Moderate or heavy freezing rain', 'Moderate or heavy sleet',
       'Moderate or heavy snow showers', 'Moderate rain', 'Moderate snow',
       'Overcast', 'Partly cloudy', 'Patchy heavy snow',
       'Patchy light drizzle', 'Patchy light snow', 'Patchy moderate snow',
       'Patchy rain possible', 'Patchy snow possible', 'Sunny', 'afternoon',
       'early_morning', 'evening', 'late_night', 'morning', 'night', 'Friday',
       'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday',
       'incident_category_Access Control', 'incident_category_Accident',
       'incident_category_Alarms', 'incident_category_Assault',
       'incident_category_Disturbance ', 'incid

In [467]:
def get_categorical_integer(row):
    if row=="Access Control":
        return int(0)
    elif row=="Accident":
        return int(1)
    elif row=="Alarms":
        return int(2)
    elif row=="Assault":
        return int(3)
    elif row=="Disturbance ":
        return int(4)
    elif row=="Elevator issue":
        return int(5)
    elif row=="Emergency":
        return int(6)
    elif row=="Equipment Misuse":
        return int(7)
    elif row=="General":
        return int(8)
    elif row=="Hazards":
        return int(9)
    elif row=="Maintenance":
        return int(10)
    elif row=="Motor Vehicle Vandalism":
        return int(11)
    elif row=="Theft":
        return int(12)
    elif row=="Tresspass":
        return int(13)
    elif row=="Unsecure element":
        return int(14)
    elif row=="Weapons found":
        return int(15)
    elif row=="none":
        return int(16)

In [468]:
dummydf = dummydf.reset_index(drop=True)
dummydf.head()

site_id  weekend  holiday  incident_time_1  Temperature  incident  \
0       14        0        0                3           32         1   
1        5        0        1                2           28         0   
2       12        0        1                3           28         0   
3      143        0        0                3           43         1   
4       15        0        0               22           43         0   

   Blizzard  Blowing snow  Clear  Cloudy  ...  incident_category_General  \
0         0             0      0       0  ...                          0   
1         0             0      0       0  ...                          0   
2         0             0      0       0  ...                          0   
3         0             0      0       0  ...                          0   
4         0             0      0       0  ...                          0   

   incident_category_Hazards  incident_category_Maintenance  \
0                          0                              0   
1                          0                              0   
2                          0                              0   
3                          0                              0   
4                          0                              0   

   incident_category_Motor Vehicle Vandalism  incident_category_Theft  \
0                                          0                        0   
1                                          0                        0   
2                                          0                        0   
3                                          0                        0   
4                                          0                        0   

   incident_category_Tresspass  incident_category_Unsecure element  \
0                            0                                   0   
1                            0                                   0   
2                            0                                   0   
3                            0                                   0   
4                            0                                   0   

   incident_category_Weapons found  incident_category_none  incident_category  
0                                0                       0       Disturbance   
1                                0                       1               none  
2                                0                       1               none  
3                                0                       0           Accident  
4                                0                       1               none  

[5 rows x 65 columns]

In [469]:

#Assign each of the crime type an integer identifier. We will input these integer identifier directly for the algorithms below.
dummydf["incident_code"]=dummydf["incident_category"].apply(get_categorical_integer)

In [470]:
dummydf.head(5)

site_id  weekend  holiday  incident_time_1  Temperature  incident  \
0       14        0        0                3           32         1   
1        5        0        1                2           28         0   
2       12        0        1                3           28         0   
3      143        0        0                3           43         1   
4       15        0        0               22           43         0   

   Blizzard  Blowing snow  Clear  Cloudy  ...  incident_category_Hazards  \
0         0             0      0       0  ...                          0   
1         0             0      0       0  ...                          0   
2         0             0      0       0  ...                          0   
3         0             0      0       0  ...                          0   
4         0             0      0       0  ...                          0   

   incident_category_Maintenance  incident_category_Motor Vehicle Vandalism  \
0                              0                                          0   
1                              0                                          0   
2                              0                                          0   
3                              0                                          0   
4                              0                                          0   

   incident_category_Theft  incident_category_Tresspass  \
0                        0                            0   
1                        0                            0   
2                        0                            0   
3                        0                            0   
4                        0                            0   

   incident_category_Unsecure element  incident_category_Weapons found  \
0                                   0                                0   
1                                   0                                0   
2                                   0                                0   
3                                   0                                0   
4                                   0                                0   

   incident_category_none  incident_category  incident_code  
0                       0       Disturbance               4  
1                       1               none             16  
2                       1               none             16  
3                       0           Accident              1  
4                       1               none             16  

[5 rows x 66 columns]

In [471]:
X = dummydf.drop(['incident_category'], axis=1)
Y = dummydf['incident_category']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

# ADABOOST MULTI-CLASS

In [183]:
from sklearn import model_selection
from sklearn.ensemble import AdaBoostClassifier
seed = 7
num_trees = 30
kfold = model_selection.KFold(n_splits=5, random_state=seed)
model_AB = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model_AB, X, Y, cv=kfold)
print(results.mean())

0.7889096340552652


In [472]:
# N-fold cross Evaluation
acc=cross_val_score(clf, X, Y, cv=5, scoring='accuracy').mean()
print("Tree Accuracy by N-fold Cross Validation:",acc)

tree = DecisionTreeClassifier()
bag = BaggingClassifier(tree, n_estimators=100, max_samples=0.8, random_state=1)
acc=cross_val_score(bag, X, Y, cv=5, scoring='accuracy').mean()
print("RandomForest Accuracy by N-fold Cross Validation:",acc)

C:\Users\SHUBHAM\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\SHUBHAM\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Tree Accuracy by N-fold Cross Validation: 0.9887510582805239
RandomForest Accuracy by N-fold Cross Validation: 0.9964285714285716


In [184]:
total_predict=len(Y_test)
correct_predict=0
for i in range(0,total_predict):
    if ((Y_test.iloc[i]==0) | (Y_test.iloc[i]==1)):
        correct_predict += 1
    if Y_test.iloc[i]==0:
        baseline_accuracy += 1
print (float(correct_predict)/total_predict)

0.0


In [185]:
## multiclass
## preciion recall
### vih- test+ predict
## random DT

# Association Rules

In [186]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import matplotlib.pyplot as plt  

In [187]:
data_asso = pd.read_csv('incidenttrain.csv')

In [188]:
data_asso.head

<bound method NDFrame.head of      site_id                       cta_sites incident_date   week_day  \
0         14       Blue Line Belmont Station     1/31/2020     Friday   
1          5          61st Street Lower Yard      1/1/2020  Wednesday   
2         12       Blue Line Addison Station      1/1/2020  Wednesday   
3        143                Skokie Rail Yard      1/2/2020   Thursday   
4         15     Blue Line Californa Station      1/2/2020   Thursday   
..       ...                             ...           ...        ...   
511       23   Blue Line Irving Park Station     3/14/2020   Saturday   
512      123        Red Line Chicago Station     3/14/2020   Saturday   
513      139  Red/Purple/Brown Lines Belmont     3/14/2020   Saturday   
514       44     Brown Line Rockwell Station     3/14/2020   Saturday   
515       45    Brown Line Southport Station     3/14/2020   Saturday   

     week_code  weekend  holiday incident_time  incident_time_1 time_category  \
0           

In [189]:
data_asso=data_asso.drop('moonrise',axis=1)
data_asso=data_asso.drop('moonset',axis=1)
data_asso=data_asso.drop('sunset',axis=1)
data_asso=data_asso.drop('sunrise',axis=1)
#data_asso=data_asso.drop('weather_description',axis=1)
data_asso=data_asso.drop('cta_sites',axis=1)
data_asso=data_asso.drop('incident_date',axis=1)
data_asso=data_asso.drop('incident_time',axis=1)
data_asso=data_asso.drop('incident_category',axis=1)
data_asso=data_asso.drop('week_code',axis=1)
data_asso=data_asso.drop('time_code',axis=1)
data_asso=data_asso.drop('weather_code',axis=1)
data_asso=data_asso.drop('incident_time_1',axis=1)
#data_asso=data_asso.drop('hours_after_sunrise',axis=1)
#data_asso=data_asso.drop('hours_to_sunset',axis=1)


In [190]:
data_asso.dtypes

site_id                  int64
week_day                object
weekend                  int64
holiday                  int64
time_category           object
atm                      int64
restaurant               int64
avgtemp                  int64
maxtemp                  int64
totalsnow              float64
sunhour                float64
Temperature              int64
weather_description     object
Humidity                 int64
cloudcover               int64
heatindex                int64
hours_after_sunrise    float64
hours_to_sunset        float64
incident                 int64
dtype: object

In [191]:
data_asso['site_id'] = pd.cut(data_asso['site_id'],30)
#data_asso['week_day'] = pd.cut(data_asso['week_day'],30)
#data_asso['week_code'] = pd.cut(data_asso['week_code'],3)
#data_asso['weekend'] = pd.cut(data_asso['weekend'],2)
#data_asso['holiday'] = pd.cut(data_asso['holiday'],2)
#data_asso['incident_time_1'] = pd.cut(data_asso['incident_time.1'],3)
#ata_asso['time_code'] = pd.cut(data_asso['time_code'],3)
#data_asso['atm'] = pd.cut(data_asso['atm'],5)
#data_asso['restaurant'] = pd.cut(data_asso['restaurant'],3)
data_asso['avgtemp'] = pd.cut(data_asso['avgtemp'],3)
data_asso['maxtemp'] = pd.cut(data_asso['maxtemp'],3)
data_asso['totalsnow'] = pd.cut(data_asso['totalsnow'],2)
data_asso['sunhour'] = pd.cut(data_asso['sunhour'],3)
data_asso['Temperature'] = pd.cut(data_asso['Temperature'],6)
data_asso['Humidity'] = pd.cut(data_asso['Humidity'],2)
#data_asso['weather_description'] = pd.cut(data_asso['weather_description'],4)
data_asso['cloudcover'] = pd.cut(data_asso['cloudcover'],2)
data_asso['heatindex'] = pd.cut(data_asso['heatindex'],2)
data_asso['hours_after_sunrise'] = pd.cut(data_asso['hours_after_sunrise'],3)
data_asso['hours_to_sunset'] = pd.cut(data_asso['hours_to_sunset'],3)
#data_asso['weather_code'] = pd.cut(data_asso['weather_code'],3)
#data_asso['incident'] = pd.cut(data_asso['incident'],2)

In [192]:
print(data_asso.dtypes)

site_id                category
week_day                 object
weekend                   int64
holiday                   int64
time_category            object
atm                       int64
restaurant                int64
avgtemp                category
maxtemp                category
totalsnow              category
sunhour                category
Temperature            category
weather_description      object
Humidity               category
cloudcover             category
heatindex              category
hours_after_sunrise    category
hours_to_sunset        category
incident                  int64
dtype: object


In [193]:
print(data_asso.head())

          site_id   week_day  weekend  holiday time_category  atm  restaurant  \
0    (11.0, 16.0]     Friday        0        0    late_night    0           0   
1     (0.85, 6.0]  Wednesday        0        1    late_night    0           0   
2    (11.0, 16.0]  Wednesday        0        1    late_night    0           0   
3  (141.0, 146.0]   Thursday        0        0    late_night    0           0   
4    (11.0, 16.0]   Thursday        0        0         night    1           1   

            avgtemp           maxtemp         totalsnow       sunhour  \
0  (22.667, 36.333]  (27.667, 41.333]  (-0.00346, 1.73]  (3.392, 6.1]   
1  (22.667, 36.333]  (27.667, 41.333]  (-0.00346, 1.73]    (6.1, 8.8]   
2  (22.667, 36.333]  (27.667, 41.333]  (-0.00346, 1.73]    (6.1, 8.8]   
3    (36.333, 50.0]    (41.333, 55.0]  (-0.00346, 1.73]  (3.392, 6.1]   
4    (36.333, 50.0]    (41.333, 55.0]  (-0.00346, 1.73]  (3.392, 6.1]   

        Temperature weather_description      Humidity     cloudcover  \
0 

In [194]:
data_asso.dtypes

site_id                category
week_day                 object
weekend                   int64
holiday                   int64
time_category            object
atm                       int64
restaurant                int64
avgtemp                category
maxtemp                category
totalsnow              category
sunhour                category
Temperature            category
weather_description      object
Humidity               category
cloudcover             category
heatindex              category
hours_after_sunrise    category
hours_to_sunset        category
incident                  int64
dtype: object

In [195]:
data_asso['site_id'] = 'site_id=' + data_asso['site_id'].astype(str)
data_asso['restaurant'] = 'restaurant=' + data_asso['restaurant'].astype(str)
data_asso['totalsnow'] = 'totalsnow' + data_asso['totalsnow'].astype(str)
data_asso['sunhour'] = 'sunhour=' + data_asso['sunhour'].astype(str)
data_asso['Humidity'] = 'Humidity' + data_asso['Humidity'].astype(str)
data_asso['cloudcover'] = 'cloudcover' + data_asso['cloudcover'].astype(str)
data_asso['hours_after_sunrise'] = 'hours_after_sunrise=' + data_asso['hours_after_sunrise'].astype(str)
data_asso['hours_to_sunset'] = 'hours_to_sunset' + data_asso['hours_to_sunset'].astype(str)
data_asso['weekend'] = 'weekend=' + data_asso['weekend'].astype(str)
#data_asso['weekday'] = 'weekday=' + data_asso['weekday'].astype(str)
data_asso['holiday'] = 'holiday' + data_asso['holiday'].astype(str)
data_asso['atm'] = 'atm' + data_asso['atm'].astype(str)
data_asso['avgtemp'] = 'avgtemp' + data_asso['avgtemp'].astype(str)
data_asso['maxtemp'] = 'maxtemp=' + data_asso['maxtemp'].astype(str)
data_asso['Temperature'] = 'Temperature' + data_asso['Temperature'].astype(str)
data_asso['heatindex'] = 'heatindex' + data_asso['heatindex'].astype(str)
data_asso['incident'] = 'incident=' + data_asso['incident'].astype(str)


In [196]:
print(data_asso.dtypes)

site_id                object
week_day               object
weekend                object
holiday                object
time_category          object
atm                    object
restaurant             object
avgtemp                object
maxtemp                object
totalsnow              object
sunhour                object
Temperature            object
weather_description    object
Humidity               object
cloudcover             object
heatindex              object
hours_after_sunrise    object
hours_to_sunset        object
incident               object
dtype: object


In [197]:
print(data_asso.head())

                  site_id   week_day    weekend   holiday time_category   atm  \
0    site_id=(11.0, 16.0]     Friday  weekend=0  holiday0    late_night  atm0   
1     site_id=(0.85, 6.0]  Wednesday  weekend=0  holiday1    late_night  atm0   
2    site_id=(11.0, 16.0]  Wednesday  weekend=0  holiday1    late_night  atm0   
3  site_id=(141.0, 146.0]   Thursday  weekend=0  holiday0    late_night  atm0   
4    site_id=(11.0, 16.0]   Thursday  weekend=0  holiday0         night  atm1   

     restaurant                  avgtemp                   maxtemp  \
0  restaurant=0  avgtemp(22.667, 36.333]  maxtemp=(27.667, 41.333]   
1  restaurant=0  avgtemp(22.667, 36.333]  maxtemp=(27.667, 41.333]   
2  restaurant=0  avgtemp(22.667, 36.333]  maxtemp=(27.667, 41.333]   
3  restaurant=0    avgtemp(36.333, 50.0]    maxtemp=(41.333, 55.0]   
4  restaurant=1    avgtemp(36.333, 50.0]    maxtemp=(41.333, 55.0]   

                   totalsnow               sunhour  \
0  totalsnow(-0.00346, 1.73]  sunhour=

In [198]:

# convert data frame to lists
df_arr = data_asso.stack().groupby(level=0).apply(list).tolist()

In [199]:
data_asso.head()

site_id   week_day    weekend   holiday time_category   atm  \
0    site_id=(11.0, 16.0]     Friday  weekend=0  holiday0    late_night  atm0   
1     site_id=(0.85, 6.0]  Wednesday  weekend=0  holiday1    late_night  atm0   
2    site_id=(11.0, 16.0]  Wednesday  weekend=0  holiday1    late_night  atm0   
3  site_id=(141.0, 146.0]   Thursday  weekend=0  holiday0    late_night  atm0   
4    site_id=(11.0, 16.0]   Thursday  weekend=0  holiday0         night  atm1   

     restaurant                  avgtemp                   maxtemp  \
0  restaurant=0  avgtemp(22.667, 36.333]  maxtemp=(27.667, 41.333]   
1  restaurant=0  avgtemp(22.667, 36.333]  maxtemp=(27.667, 41.333]   
2  restaurant=0  avgtemp(22.667, 36.333]  maxtemp=(27.667, 41.333]   
3  restaurant=0    avgtemp(36.333, 50.0]    maxtemp=(41.333, 55.0]   
4  restaurant=1    avgtemp(36.333, 50.0]    maxtemp=(41.333, 55.0]   

                   totalsnow               sunhour  \
0  totalsnow(-0.00346, 1.73]  sunhour=(3.392, 6.1]   
1  totalsnow(-0.00346, 1.73]    sunhour=(6.1, 8.8]   
2  totalsnow(-0.00346, 1.73]    sunhour=(6.1, 8.8]   
3  totalsnow(-0.00346, 1.73]  sunhour=(3.392, 6.1]   
4  totalsnow(-0.00346, 1.73]  sunhour=(3.392, 6.1]   

                   Temperature weather_description              Humidity  \
0    Temperature(29.5, 37.667]            Overcast  Humidity(65.5, 99.0]   
1    Temperature(21.333, 29.5]       Partly cloudy  Humidity(65.5, 99.0]   
2    Temperature(21.333, 29.5]       Partly cloudy  Humidity(65.5, 99.0]   
3  Temperature(37.667, 45.833]       Partly cloudy  Humidity(65.5, 99.0]   
4  Temperature(37.667, 45.833]       Partly cloudy  Humidity(65.5, 99.0]   

                cloudcover               heatindex  \
0  cloudcover(50.0, 100.0]   heatindex(29.5, 54.0]   
1   cloudcover(-0.1, 50.0]  heatindex(4.951, 29.5]   
2   cloudcover(-0.1, 50.0]  heatindex(4.951, 29.5]   
3  cloudcover(50.0, 100.0]   heatindex(29.5, 54.0]   
4  cloudcover(50.0, 100.0]   heatindex(29.5, 54.0]   

                   hours_after_sunrise                 hours_to_sunset  \
0  hours_after_sunrise=(-8.325, 0.107]   hours_to_sunset(10.623, 18.9]   
1  hours_after_sunrise=(-8.325, 0.107]   hours_to_sunset(10.623, 18.9]   
2  hours_after_sunrise=(-8.325, 0.107]   hours_to_sunset(10.623, 18.9]   
3  hours_after_sunrise=(-8.325, 0.107]   hours_to_sunset(10.623, 18.9]   
4   hours_after_sunrise=(8.513, 16.92]  hours_to_sunset(-5.955, 2.347]   

     incident  
0  incident=1  
1  incident=0  
2  incident=0  
3  incident=1  
4  incident=0

In [200]:
#Rules
import numpy as np
import scipy as sp
import pandas as pd
from IPython.display import display, HTML

# Encode lists to transactions
te = TransactionEncoder()
df_transactions = te.fit_transform(df_arr)
# covnert the values to booleans: TRUE and FALSE
df_rules = pd.DataFrame(df_transactions,columns=te.columns_)
display('Data Example',HTML(df_rules.head(5).to_html()))


frequent_itemsets = apriori(df_rules, min_support=0.50, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
display('Rules',HTML(rules.to_html()))

'Data Example'

'Rules'

In [201]:
incident=rules[rules['consequents'] == {'incident=1'}]

In [202]:
incident

antecedents   consequents  \
9                               (Humidity(65.5, 99.0])  (incident=1)   
50                             (heatindex(29.5, 54.0])  (incident=1)   
59                                          (holiday0)  (incident=1)   
69                          (maxtemp=(27.667, 41.333])  (incident=1)   
70                         (totalsnow(-0.00346, 1.73])  (incident=1)   
73                                         (weekend=0)  (incident=1)   
129                   (Humidity(65.5, 99.0], holiday0)  (incident=1)   
153  (totalsnow(-0.00346, 1.73], Humidity(65.5, 99.0])  (incident=1)   
250               (maxtemp=(27.667, 41.333], holiday0)  (incident=1)   
255              (totalsnow(-0.00346, 1.73], holiday0)  (incident=1)   
261                              (weekend=0, holiday0)  (incident=1)   
285             (totalsnow(-0.00346, 1.73], weekend=0)  (incident=1)   
353  (Humidity(65.5, 99.0], totalsnow(-0.00346, 1.7...  (incident=1)   
451   (totalsnow(-0.00346, 1.73], weekend=0, holiday0)  (incident=1)   

     antecedent support  consequent support   support  confidence      lift  \
9              0.782946             0.71124  0.569767    0.727723  1.023174   
50             0.720930             0.71124  0.505814    0.701613  0.986464   
59             0.968992             0.71124  0.691860    0.714000  1.003880   
69             0.720930             0.71124  0.519380    0.720430  1.012921   
70             0.941860             0.71124  0.666667    0.707819  0.995190   
73             0.759690             0.71124  0.554264    0.729592  1.025802   
129            0.751938             0.71124  0.550388    0.731959  1.029130   
153            0.724806             0.71124  0.525194    0.724599  1.018782   
250            0.689922             0.71124  0.500000    0.724719  1.018951   
255            0.910853             0.71124  0.647287    0.710638  0.999154   
261            0.728682             0.71124  0.534884    0.734043  1.032060   
285            0.734496             0.71124  0.534884    0.728232  1.023890   
353            0.693798             0.71124  0.505814    0.729050  1.025041   
451            0.703488             0.71124  0.515504    0.732782  1.030288   

     leverage  conviction  
9    0.012905    1.060536  
50  -0.006941    0.967735  
59   0.002674    1.009649  
69   0.006625    1.032871  
70  -0.003222    0.988290  
73   0.013941    1.067866  
129  0.015579    1.077296  
153  0.009682    1.048506  
250  0.009299    1.048964  
255 -0.000548    0.997920  
261  0.016616    1.085736  
285  0.012480    1.062524  
353  0.012357    1.065732  
451  0.015155    1.080616

# MULTI CLASS TYPES

In [301]:
dummydf.columns

Index(['site_id', 'weekend', 'holiday', 'incident_time_1', 'Temperature',
       'incident', 'Blizzard', 'Blowing snow', 'Clear', 'Cloudy', 'Heavy rain',
       'Heavy snow', 'Ice pellets', 'Light drizzle', 'Light rain',
       'Light rain shower', 'Light sleet', 'Light snow', 'Light snow showers',
       'Mist', 'Moderate or heavy freezing rain', 'Moderate or heavy sleet',
       'Moderate or heavy snow showers', 'Moderate rain', 'Moderate snow',
       'Overcast', 'Partly cloudy', 'Patchy heavy snow',
       'Patchy light drizzle', 'Patchy light snow', 'Patchy moderate snow',
       'Patchy rain possible', 'Patchy snow possible', 'Sunny', 'afternoon',
       'early_morning', 'evening', 'late_night', 'morning', 'night', 'Friday',
       'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday',
       'incident_category_Access Control', 'incident_category_Accident',
       'incident_category_Alarms', 'incident_category_Assault',
       'incident_category_Disturbance ', 'incid

In [486]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import matplotlib.pyplot as plt  
dummydf1=dummydf.copy(deep=True)

In [487]:
dummydf1

site_id  weekend  holiday  incident_time_1  Temperature  incident  \
0         14        0        0                3           32         1   
1          5        0        1                2           28         0   
2         12        0        1                3           28         0   
3        143        0        0                3           43         1   
4         15        0        0               22           43         0   
..       ...      ...      ...              ...          ...       ...   
511       23        1        0                1           39         1   
512      123        1        0                3           37         1   
513      139        1        0               23           32         1   
514       44        1        0                8           36         0   
515       45        1        0                9           36         0   

     Blizzard  Blowing snow  Clear  Cloudy  ...  incident_category_Hazards  \
0           0             0      0       0  ...                          0   
1           0             0      0       0  ...                          0   
2           0             0      0       0  ...                          0   
3           0             0      0       0  ...                          0   
4           0             0      0       0  ...                          0   
..        ...           ...    ...     ...  ...                        ...   
511         0             0      0       0  ...                          0   
512         0             0      0       1  ...                          0   
513         0             0      0       0  ...                          0   
514         0             0      0       0  ...                          0   
515         0             0      0       0  ...                          0   

     incident_category_Maintenance  incident_category_Motor Vehicle Vandalism  \
0                                0                                          0   
1                                0                                          0   
2                                0                                          0   
3                                0                                          0   
4                                0                                          0   
..                             ...                                        ...   
511                              0                                          0   
512                              0                                          0   
513                              0                                          0   
514                              0                                          0   
515                              0                                          0   

     incident_category_Theft  incident_category_Tresspass  \
0                          0                            0   
1                          0                            0   
2                          0                            0   
3                          0                            0   
4                          0                            0   
..                       ...                          ...   
511                        0                            0   
512                        0                            0   
513                        0                            0   
514                        0                            0   
515                        0                            0   

     incident_category_Unsecure element  incident_category_Weapons found  \
0                                     0                                0   
1                                     0                                0   
2                                     0                                0   
3                                     0                                0   
4                                     0                                0   
..               

In [488]:
dummydf2=dummydf1.drop(['weekend', 'holiday', 'incident_time_1', 'Temperature', 'Blizzard', 'Blowing snow', 'Clear', 'Cloudy', 'Heavy rain',
       'Heavy snow', 'Ice pellets', 'Light drizzle', 'Light rain',
       'Light rain shower', 'Light sleet', 'Light snow', 'Light snow showers',
       'Mist', 'Moderate or heavy freezing rain', 'Moderate or heavy sleet',
       'Moderate or heavy snow showers', 'Moderate rain', 'Moderate snow',
       'Overcast', 'Partly cloudy', 'Patchy heavy snow',
       'Patchy light drizzle', 'Patchy light snow', 'Patchy moderate snow',
       'Patchy rain possible', 'Patchy snow possible', 'Sunny', 'afternoon',
       'early_morning', 'evening', 'late_night', 'morning', 'night', 'Friday',
       'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday','incident_category','incident_code'],axis=1)

In [489]:
dummydf2.columns

Index(['site_id', 'incident', 'incident_category_Access Control',
       'incident_category_Accident', 'incident_category_Alarms',
       'incident_category_Assault', 'incident_category_Disturbance ',
       'incident_category_Elevator issue', 'incident_category_Emergency',
       'incident_category_Equipment Misuse', 'incident_category_General',
       'incident_category_Hazards', 'incident_category_Maintenance',
       'incident_category_Motor Vehicle Vandalism', 'incident_category_Theft',
       'incident_category_Tresspass', 'incident_category_Unsecure element',
       'incident_category_Weapons found', 'incident_category_none'],
      dtype='object')

In [490]:
dummydf2['site_id'] = pd.cut(dummydf2['site_id'],30)


In [491]:
dummydf2['site_id'] = 'site_id=' + dummydf2['site_id'].astype(str)
dummydf2['incident_category_Access Control'] = 'incident_category_Access Control=' + dummydf2['incident_category_Access Control'].astype(str)
dummydf2['incident_category_Accident'] = 'incident_category_Accident=' + dummydf2['incident_category_Accident'].astype(str)
dummydf2['incident_category_Alarms'] = 'incident_category_Alarms='+ dummydf2['incident_category_Alarms'].astype(str)
dummydf2['incident_category_Assault'] = 'incident_category_Assault='+ dummydf2['incident_category_Assault'].astype(str)
dummydf2['incident_category_Disturbance '] ='incident_category_Disturbance ='+ dummydf2['incident_category_Disturbance '].astype(str)
dummydf2['incident_category_Elevator issue'] ='incident_category_Elevator issue='+ dummydf2['incident_category_Elevator issue'].astype(str)
dummydf2['incident_category_Emergency'] = 'incident_category_Emergency='+ dummydf2['incident_category_Emergency'].astype(str)
dummydf2['incident_category_Equipment Misuse'] = 'incident_category_Equipment Misuse='+ dummydf2['incident_category_Equipment Misuse'].astype(str)
dummydf2['incident_category_General'] = 'incident_category_General='+ dummydf2['incident_category_General'].astype(str)
dummydf2['incident_category_Hazards'] = 'incident_category_Hazards='+ dummydf2['incident_category_Hazards'].astype(str)
dummydf2['incident_category_Maintenance'] = 'incident_category_Maintenance='+ dummydf2['incident_category_Maintenance'].astype(str)
dummydf2['incident_category_Motor Vehicle Vandalism'] = 'incident_category_Motor Vehicle Vandalism='+ dummydf2['incident_category_Motor Vehicle Vandalism'].astype(str)
dummydf2['incident_category_Theft'] = 'incident_category_Theft='+ dummydf2['incident_category_Theft'].astype(str)
dummydf2['incident_category_Tresspass'] = 'incident_category_Tresspass='+ dummydf2['incident_category_Tresspass'].astype(str)
dummydf2['incident_category_Unsecure element'] = 'incident_category_Unsecure element='+ dummydf2['incident_category_Unsecure element'].astype(str)
dummydf2['incident_category_Weapons found'] = 'incident_category_Weapons found='+ dummydf2['incident_category_Weapons found'].astype(str)
dummydf2['incident_category_none'] = 'incident_category_none='+ dummydf2['incident_category_none'].astype(str)



In [492]:
dummydf2.dtypes

site_id                                      object
incident                                      int64
incident_category_Access Control             object
incident_category_Accident                   object
incident_category_Alarms                     object
incident_category_Assault                    object
incident_category_Disturbance                object
incident_category_Elevator issue             object
incident_category_Emergency                  object
incident_category_Equipment Misuse           object
incident_category_General                    object
incident_category_Hazards                    object
incident_category_Maintenance                object
incident_category_Motor Vehicle Vandalism    object
incident_category_Theft                      object
incident_category_Tresspass                  object
incident_category_Unsecure element           object
incident_category_Weapons found              object
incident_category_none                       object
dtype: objec

In [493]:
dummydf2.head()

site_id  incident    incident_category_Access Control  \
0    site_id=(11.0, 16.0]         1  incident_category_Access Control=0   
1     site_id=(0.85, 6.0]         0  incident_category_Access Control=0   
2    site_id=(11.0, 16.0]         0  incident_category_Access Control=0   
3  site_id=(141.0, 146.0]         1  incident_category_Access Control=0   
4    site_id=(11.0, 16.0]         0  incident_category_Access Control=0   

     incident_category_Accident    incident_category_Alarms  \
0  incident_category_Accident=0  incident_category_Alarms=0   
1  incident_category_Accident=0  incident_category_Alarms=0   
2  incident_category_Accident=0  incident_category_Alarms=0   
3  incident_category_Accident=1  incident_category_Alarms=0   
4  incident_category_Accident=0  incident_category_Alarms=0   

     incident_category_Assault    incident_category_Disturbance   \
0  incident_category_Assault=0  incident_category_Disturbance =1   
1  incident_category_Assault=0  incident_category_Disturbance =0   
2  incident_category_Assault=0  incident_category_Disturbance =0   
3  incident_category_Assault=0  incident_category_Disturbance =0   
4  incident_category_Assault=0  incident_category_Disturbance =0   

     incident_category_Elevator issue    incident_category_Emergency  \
0  incident_category_Elevator issue=0  incident_category_Emergency=0   
1  incident_category_Elevator issue=0  incident_category_Emergency=0   
2  incident_category_Elevator issue=0  incident_category_Emergency=0   
3  incident_category_Elevator issue=0  incident_category_Emergency=0   
4  incident_category_Elevator issue=0  incident_category_Emergency=0   

     incident_category_Equipment Misuse    incident_category_General  \
0  incident_category_Equipment Misuse=0  incident_category_General=0   
1  incident_category_Equipment Misuse=0  incident_category_General=0   
2  incident_category_Equipment Misuse=0  incident_category_General=0   
3  incident_category_Equipment Misuse=0  incident_category_General=0   
4  incident_category_Equipment Misuse=0  incident_category_General=0   

     incident_category_Hazards    incident_category_Maintenance  \
0  incident_category_Hazards=0  incident_category_Maintenance=0   
1  incident_category_Hazards=0  incident_category_Maintenance=0   
2  incident_category_Hazards=0  incident_category_Maintenance=0   
3  incident_category_Hazards=0  incident_category_Maintenance=0   
4  incident_category_Hazards=0  incident_category_Maintenance=0   

     incident_category_Motor Vehicle Vandalism    incident_category_Theft  \
0  incident_category_Motor Vehicle Vandalism=0  incident_category_Theft=0   
1  incident_category_Motor Vehicle Vandalism=0  incident_category_Theft=0   
2  incident_category_Motor Vehicle Vandalism=0  incident_category_Theft=0   
3  incident_category_Motor Vehicle Vandalism=0  incident_category_Theft=0   
4  incident_category_Motor Vehicle Vandalism=0  incident_category_Theft=0   

     incident_category_Tresspass    incident_category_Unsecure element  \
0  incident_category_Tresspass=0  incident_category_Unsecure element=0   
1  incident_category_Tresspass=0  incident_category_Unsecure element=0   
2  incident_category_Tresspass=0  incident_category_Unsecure element=0   
3  incident_category_Tresspass=0  incident_category_Unsecure element=0   
4  incident_category_Tresspass=0  incident_category_Unsecure element=0   

     incident_category_Weapons found    incident_category_none  
0  incident_category_Weapons found=0  incident_category_none=0  
1  incident_category_Weapons found=0  incident_category_none=1  
2  incident_category_Weapons found=0  incident_category_none=1  
3  incident_category_Weapons found=0  incident_category_none=0  
4  incident_category_Weapons found=0  incident_category_none=1

In [494]:
dummydf2.dtypes

site_id                                      object
incident                                      int64
incident_category_Access Control             object
incident_category_Accident                   object
incident_category_Alarms                     object
incident_category_Assault                    object
incident_category_Disturbance                object
incident_category_Elevator issue             object
incident_category_Emergency                  object
incident_category_Equipment Misuse           object
incident_category_General                    object
incident_category_Hazards                    object
incident_category_Maintenance                object
incident_category_Motor Vehicle Vandalism    object
incident_category_Theft                      object
incident_category_Tresspass                  object
incident_category_Unsecure element           object
incident_category_Weapons found              object
incident_category_none                       object
dtype: objec

In [495]:
print(dummydf2.head())

                  site_id  incident    incident_category_Access Control  \
0    site_id=(11.0, 16.0]         1  incident_category_Access Control=0   
1     site_id=(0.85, 6.0]         0  incident_category_Access Control=0   
2    site_id=(11.0, 16.0]         0  incident_category_Access Control=0   
3  site_id=(141.0, 146.0]         1  incident_category_Access Control=0   
4    site_id=(11.0, 16.0]         0  incident_category_Access Control=0   

     incident_category_Accident    incident_category_Alarms  \
0  incident_category_Accident=0  incident_category_Alarms=0   
1  incident_category_Accident=0  incident_category_Alarms=0   
2  incident_category_Accident=0  incident_category_Alarms=0   
3  incident_category_Accident=1  incident_category_Alarms=0   
4  incident_category_Accident=0  incident_category_Alarms=0   

     incident_category_Assault    incident_category_Disturbance   \
0  incident_category_Assault=0  incident_category_Disturbance =1   
1  incident_category_Assault=0  in

In [497]:
# convert data frame to lists
dummy_arr = dummydf2.stack().groupby(level=0).apply.list().tolist()

AttributeError: 'SeriesGroupBy' object has no attribute 'spply'

In [484]:
dummydf2.head()

site_id  incident    incident_category_Access Control  \
0    site_id=(11.0, 16.0]         1  incident_category_Access Control=0   
1     site_id=(0.85, 6.0]         0  incident_category_Access Control=0   
2    site_id=(11.0, 16.0]         0  incident_category_Access Control=0   
3  site_id=(141.0, 146.0]         1  incident_category_Access Control=0   
4    site_id=(11.0, 16.0]         0  incident_category_Access Control=0   

     incident_category_Accident    incident_category_Alarms  \
0  incident_category_Accident=0  incident_category_Alarms=0   
1  incident_category_Accident=0  incident_category_Alarms=0   
2  incident_category_Accident=0  incident_category_Alarms=0   
3  incident_category_Accident=1  incident_category_Alarms=0   
4  incident_category_Accident=0  incident_category_Alarms=0   

     incident_category_Assault    incident_category_Disturbance   \
0  incident_category_Assault=0  incident_category_Disturbance =1   
1  incident_category_Assault=0  incident_category_Disturbance =0   
2  incident_category_Assault=0  incident_category_Disturbance =0   
3  incident_category_Assault=0  incident_category_Disturbance =0   
4  incident_category_Assault=0  incident_category_Disturbance =0   

     incident_category_Elevator issue    incident_category_Emergency  \
0  incident_category_Elevator issue=0  incident_category_Emergency=0   
1  incident_category_Elevator issue=0  incident_category_Emergency=0   
2  incident_category_Elevator issue=0  incident_category_Emergency=0   
3  incident_category_Elevator issue=0  incident_category_Emergency=0   
4  incident_category_Elevator issue=0  incident_category_Emergency=0   

     incident_category_Equipment Misuse    incident_category_General  \
0  incident_category_Equipment Misuse=0  incident_category_General=0   
1  incident_category_Equipment Misuse=0  incident_category_General=0   
2  incident_category_Equipment Misuse=0  incident_category_General=0   
3  incident_category_Equipment Misuse=0  incident_category_General=0   
4  incident_category_Equipment Misuse=0  incident_category_General=0   

     incident_category_Hazards    incident_category_Maintenance  \
0  incident_category_Hazards=0  incident_category_Maintenance=0   
1  incident_category_Hazards=0  incident_category_Maintenance=0   
2  incident_category_Hazards=0  incident_category_Maintenance=0   
3  incident_category_Hazards=0  incident_category_Maintenance=0   
4  incident_category_Hazards=0  incident_category_Maintenance=0   

     incident_category_Motor Vehicle Vandalism    incident_category_Theft  \
0  incident_category_Motor Vehicle Vandalism=0  incident_category_Theft=0   
1  incident_category_Motor Vehicle Vandalism=0  incident_category_Theft=0   
2  incident_category_Motor Vehicle Vandalism=0  incident_category_Theft=0   
3  incident_category_Motor Vehicle Vandalism=0  incident_category_Theft=0   
4  incident_category_Motor Vehicle Vandalism=0  incident_category_Theft=0   

     incident_category_Tresspass    incident_category_Unsecure element  \
0  incident_category_Tresspass=0  incident_category_Unsecure element=0   
1  incident_category_Tresspass=0  incident_category_Unsecure element=0   
2  incident_category_Tresspass=0  incident_category_Unsecure element=0   
3  incident_category_Tresspass=0  incident_category_Unsecure element=0   
4  incident_category_Tresspass=0  incident_category_Unsecure element=0   

     incident_category_Weapons found    incident_category_none  
0  incident_category_Weapons found=0  incident_category_none=0  
1  incident_category_Weapons found=0  incident_category_none=1  
2  incident_category_Weapons found=0  incident_category_none=1  
3  incident_category_Weapons found=0  incident_category_none=0  
4  incident_category_Weapons found=0  incident_category_none=1

In [485]:
#Rules
import numpy as np
import scipy as sp
import pandas as pd
from IPython.display import display, HTML

# Encode lists to transactions
te = TransactionEncoder()
df_transactions = te.fit_transform(dummy_arr)
# covnert the values to booleans: TRUE and FALSE
df_rules = pd.DataFrame(df_transactions,columns=te.columns_)
display('Data Example',HTML(df_rules.head(5).to_html()))

 
frequent_itemsets =apriori(df_rules, min_support=0.50, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
display('Rules',HTML(rules.to_html()))

TypeError: '<' not supported between instances of 'str' and 'int'